In [1]:
import json
import pandas as pd
from flask import Flask
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

In [2]:
conteudos = pd.read_excel('conteudos.xlsx', sheet_name='conteudos')
dataset = pd.read_excel('conteudos.xlsx', sheet_name='consumidos')
investidos = pd.read_excel('conteudos.xlsx', sheet_name='investidos')

In [3]:
t = dataset.T.drop(['usuario', 'perfil'], axis=0)
df = []
for coluna in t:
    lista = []
    for i in t[coluna]:
        if type(i) == str:
            lista.append(i)
    df.append(lista)

te = TransactionEncoder()
te_ary = te.fit(df).transform(df)
transf = pd.DataFrame(te_ary, columns=te.columns_)

In [4]:
frequent_itemsets = apriori(transf, min_support=0.3, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
itens_associados = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

In [5]:
conteudo = 'como-investir-no-tesouro-direto-conheca-seus-tipos-e-as-diferencas-entre-cada-um'

In [6]:
def recomendacao(conteudo):
    for i in range(len(itens_associados)):
        if conteudo in itens_associados['antecedents'].loc[i]:
            recomendar = str(itens_associados['consequents'].loc[i])
            recomendar = recomendar.replace('frozenset', 'recomendar')
            retorno = {conteudo: recomendar}
            break
    return json.dumps(retorno)

recomendacao(conteudo)

'{"como-investir-no-tesouro-direto-conheca-seus-tipos-e-as-diferencas-entre-cada-um": "recomendar({\'como-e-uma-reuniao-do-copom-e-qual-a-importancia-da-taxa-selic\'})"}'

In [10]:
def recomendar_investimento():
    for i in range(len(investidos)):
        if (investidos['percentil_recomendado'].loc[i] - investidos['percentil_investido'].loc[i]) > 1:
            #cont = recomendacao(dataset['conteudo1'].loc[i])
            investimento_recomendar = investidos['investimento'].loc[i]
        #return json.dumps({investimento_recomendar})
        return investimento_recomendar
recomendar_investimento()  

'renda_fixa'